In [1]:
import os
import pandas as pd
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("CoffeeData") \
    .master("local[*]") \
    .getOrCreate()

spark

In [3]:
import pandas as pd

df_pd = pd.read_csv(
    r"C:\Users\HP\Desktop\Coffee\data\cleaned\coffee-cleaned.csv",
    sep=';',             
    encoding='utf-8',   
    header =0
)
print(df_pd.shape)
df_pd.head()

(2215, 14)


,coffee_name,total_score,roaster_location,coffee_origin,roast_level,est._price,agtron_ground,agtron_roast,price_per_100g,price_per_ounce,origin_country,clean_roaster_location,roaster_country,origin_country_clean
0,Colombia Finca Campo Hermosa,94.0,"Cleveland, Tennessee","Quindio Department, Colombia",Light,$29.99/8 ounces,62.0,82.0,13.22,29.990000,Colombia,"Cleveland, Tennessee",USA,Colombia
1,Colombia Finca La Sirena Mango Co-Ferment,94.0,"Cleveland, Tennessee","Quindio Department, Colombia",Light,$22.99/8 ounces,63.0,87.0,10.14,22.990000,Colombia,"Cleveland, Tennessee",USA,Colombia
2,In Bloom,94.0,"Jersey City, New Jersey","Colombia, Ethiopia",Light,$25.00/250 grams,64.0,88.0,10.00,0.881834,Ethiopia,"Jersey, New Jersey",USA,Ethiopia
3,Ethiopia Washed Kaffa Gimbo Lot Rich Espresso,96.0,"Chia-Yi, Taiwan","Gimbo, Kaffa Province, Ethiopia",Medium Light,NT $250/8 ounces,55.0,77.0,110.23,250.000000,Ethiopia,"Chia-Yi, Taiwan",Taiwan,Ethiopia
4,Ethiopia Natural Gute Bona,95.0,"Chia-Yi, Taiwan","Sidamo growing region, southern Ethiopia",Medium Light,NT $400/8 ounces,62.0,78.0,176.37,400.000000,Ethiopia,"Chia-Yi, Taiwan",Taiwan,Ethiopia


In [4]:
#df_pyspark = spark.read.csv('coffee-cleaned.csv', sep = ';')
df = spark.read.option("header", True) \
               .option("sep", ";") \
               .option("encoding", "UTF-8") \
               .csv("coffee-cleaned.csv")

df.printSchema()
type(df)

root
 |-- coffee_name: string (nullable = true)
 |-- total_score: string (nullable = true)
 |-- roaster_location: string (nullable = true)
 |-- coffee_origin: string (nullable = true)
 |-- roast_level: string (nullable = true)
 |-- est._price: string (nullable = true)
 |-- agtron_ground: string (nullable = true)
 |-- agtron_roast: string (nullable = true)
 |-- price_per_100g: string (nullable = true)
 |-- price_per_ounce: string (nullable = true)
 |-- origin_country: string (nullable = true)
 |-- clean_roaster_location: string (nullable = true)
 |-- roaster_country: string (nullable = true)
 |-- origin_country_clean: string (nullable = true)



pyspark.sql.classic.dataframe.DataFrame

In [5]:
df = df.drop("roaster_location", "est._price", "price_per_ounce", "origin_country", "clean_roaster_location")
df.show()

+--------------------+-----------+--------------------+------------+-------------+------------+--------------+---------------+--------------------+
|         coffee_name|total_score|       coffee_origin| roast_level|agtron_ground|agtron_roast|price_per_100g|roaster_country|origin_country_clean|
+--------------------+-----------+--------------------+------------+-------------+------------+--------------+---------------+--------------------+
|Colombia Finca Ca...|         94|Quindio Departmen...|       Light|           62|          82|         13.22|            USA|            Colombia|
|Colombia Finca La...|         94|Quindio Departmen...|       Light|           63|          87|         10.14|            USA|            Colombia|
|            In Bloom|         94|  Colombia, Ethiopia|       Light|           64|          88|            10|            USA|            Ethiopia|
|Ethiopia Washed K...|         96|Gimbo, Kaffa Prov...|Medium Light|           55|          77|        110.23|  

In [6]:
df = df.drop("roaster_location", "est._price", "price_per_ounce", "origin_country", "clean_roaster_location")
df.show()

+--------------------+-----------+--------------------+------------+-------------+------------+--------------+---------------+--------------------+
|         coffee_name|total_score|       coffee_origin| roast_level|agtron_ground|agtron_roast|price_per_100g|roaster_country|origin_country_clean|
+--------------------+-----------+--------------------+------------+-------------+------------+--------------+---------------+--------------------+
|Colombia Finca Ca...|         94|Quindio Departmen...|       Light|           62|          82|         13.22|            USA|            Colombia|
|Colombia Finca La...|         94|Quindio Departmen...|       Light|           63|          87|         10.14|            USA|            Colombia|
|            In Bloom|         94|  Colombia, Ethiopia|       Light|           64|          88|            10|            USA|            Ethiopia|
|Ethiopia Washed K...|         96|Gimbo, Kaffa Prov...|Medium Light|           55|          77|        110.23|  

In [7]:
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType

df = df.withColumn("price_per_100g", col("price_per_100g").cast(DoubleType()))
df.show()

+--------------------+-----------+--------------------+------------+-------------+------------+--------------+---------------+--------------------+
|         coffee_name|total_score|       coffee_origin| roast_level|agtron_ground|agtron_roast|price_per_100g|roaster_country|origin_country_clean|
+--------------------+-----------+--------------------+------------+-------------+------------+--------------+---------------+--------------------+
|Colombia Finca Ca...|         94|Quindio Departmen...|       Light|           62|          82|         13.22|            USA|            Colombia|
|Colombia Finca La...|         94|Quindio Departmen...|       Light|           63|          87|         10.14|            USA|            Colombia|
|            In Bloom|         94|  Colombia, Ethiopia|       Light|           64|          88|          10.0|            USA|            Ethiopia|
|Ethiopia Washed K...|         96|Gimbo, Kaffa Prov...|Medium Light|           55|          77|        110.23|  

In [8]:
### описова статистика 
df.groupBy('origin_country_clean').mean('price_per_100g').show()
### India is NULL. Why?
### NULL country

+--------------------+-------------------+
|origin_country_clean|avg(price_per_100g)|
+--------------------+-------------------+
|               Yemen| 122.30839999999998|
|              Rwanda|  68.84200000000001|
|              Taiwan| 403.74500000000006|
|                NULL|  40.40850490196077|
|             Ecuador| 294.39928571428567|
|           Nicaragua|  8.578333333333333|
|                Peru|            168.382|
|       United States| 22.114814814814817|
|               China|              88.11|
|               India|               NULL|
|Democratic Republ...|  5.578333333333333|
|              Panama| 241.71649681528652|
|              Mexico|             99.837|
|           Indonesia|  34.61774193548386|
|           Guatemala| 147.21472972972973|
|            Honduras|            72.6825|
|            Ethiopia| 109.51148423005563|
|              Brazil|  391.9822222222222|
|               Kenya|  81.24762499999993|
|            Tanzania|  36.31058823529412|
+----------

In [9]:
###India. All records
df.filter(col("origin_country_clean") == "India").show()
### No info about price

+--------------------+-----------+--------------------+-----------+-------------+------------+--------------+---------------+--------------------+
|         coffee_name|total_score|       coffee_origin|roast_level|agtron_ground|agtron_roast|price_per_100g|roaster_country|origin_country_clean|
+--------------------+-----------+--------------------+-----------+-------------+------------+--------------+---------------+--------------------+
|India AA Elkhill ...|         95|Coorg growing reg...|      Light|           66|          85|          NULL|            USA|               India|
|   S.O. India Sitara|         94|      Southern India|     Medium|           45|          67|          NULL|            USA|               India|
+--------------------+-----------+--------------------+-----------+-------------+------------+--------------+---------------+--------------------+



In [10]:
df_cl = df.na.drop(how="any")
df_cl.show()
num_rows = df_cl.count()
num_cols = len(df_cl.columns)
print((num_rows, num_cols))

+--------------------+-----------+--------------------+------------+-------------+------------+--------------+---------------+--------------------+
|         coffee_name|total_score|       coffee_origin| roast_level|agtron_ground|agtron_roast|price_per_100g|roaster_country|origin_country_clean|
+--------------------+-----------+--------------------+------------+-------------+------------+--------------+---------------+--------------------+
|Colombia Finca Ca...|         94|Quindio Departmen...|       Light|           62|          82|         13.22|            USA|            Colombia|
|Colombia Finca La...|         94|Quindio Departmen...|       Light|           63|          87|         10.14|            USA|            Colombia|
|            In Bloom|         94|  Colombia, Ethiopia|       Light|           64|          88|          10.0|            USA|            Ethiopia|
|Ethiopia Washed K...|         96|Gimbo, Kaffa Prov...|Medium Light|           55|          77|        110.23|  

In [11]:
df_cl.groupBy('origin_country_clean') \
       .mean('price_per_100g') \
       .orderBy('avg(price_per_100g)', ascending=False) \
       .show()

+--------------------+-------------------+
|origin_country_clean|avg(price_per_100g)|
+--------------------+-------------------+
|              Brazil|  467.0728571428571|
|              Taiwan| 403.74500000000006|
|             Ecuador| 294.39928571428567|
|              Panama| 241.71649681528652|
|                Peru|            168.382|
|           Guatemala| 147.21472972972973|
|               Yemen| 129.99173913043478|
|            Ethiopia| 110.31951401869155|
|              Mexico|             99.837|
|               China|              88.11|
|               Kenya|  81.24762499999993|
|            Honduras|            72.6825|
|          Costa Rica|  69.06649350649352|
|              Rwanda|  68.84200000000001|
|            Colombia|  67.43263513513516|
|            Tanzania|  38.47562500000001|
|           Indonesia|  34.61774193548386|
|         El Salvador| 30.712500000000006|
|       United States| 22.114814814814817|
|           Nicaragua|  8.578333333333333|
+----------

In [12]:
df_cl.groupBy('origin_country_clean') \
       .max('price_per_100g') \
       .orderBy('max(price_per_100g)', ascending=False) \
       .show()

+--------------------+-------------------+
|origin_country_clean|max(price_per_100g)|
+--------------------+-------------------+
|              Panama|             6000.0|
|             Ecuador|             4000.0|
|              Brazil|             2980.0|
|            Ethiopia|            2557.32|
|            Colombia|             1680.0|
|              Taiwan|             1200.0|
|           Guatemala|             1058.2|
|                Peru|              800.0|
|               Yemen|             793.65|
|               Kenya|             528.63|
|          Costa Rica|             509.25|
|              Mexico|              498.0|
|         El Salvador|             440.92|
|            Tanzania|              399.0|
|              Rwanda|             352.73|
|           Indonesia|             308.64|
|            Honduras|             154.19|
|               China|              88.11|
|       United States|              70.55|
|           Nicaragua|              26.41|
+----------

In [13]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="roast_level", outputCol="roast_level_index")
df_r = indexer.fit(df).transform(df)
df_r.show()

+--------------------+-----------+--------------------+------------+-------------+------------+--------------+---------------+--------------------+-----------------+
|         coffee_name|total_score|       coffee_origin| roast_level|agtron_ground|agtron_roast|price_per_100g|roaster_country|origin_country_clean|roast_level_index|
+--------------------+-----------+--------------------+------------+-------------+------------+--------------+---------------+--------------------+-----------------+
|Colombia Finca Ca...|         94|Quindio Departmen...|       Light|           62|          82|         13.22|            USA|            Colombia|              1.0|
|Colombia Finca La...|         94|Quindio Departmen...|       Light|           63|          87|         10.14|            USA|            Colombia|              1.0|
|            In Bloom|         94|  Colombia, Ethiopia|       Light|           64|          88|          10.0|            USA|            Ethiopia|              1.0|
|Eth

In [14]:
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType

df_r = df_r.withColumn("total_score", col("total_score").cast(DoubleType()))
df_r = df_r.withColumn("agtron_ground", col("agtron_ground").cast(DoubleType()))
df_r = df_r.withColumn("agtron_roast", col("agtron_roast").cast(DoubleType()))

df_r.show()

+--------------------+-----------+--------------------+------------+-------------+------------+--------------+---------------+--------------------+-----------------+
|         coffee_name|total_score|       coffee_origin| roast_level|agtron_ground|agtron_roast|price_per_100g|roaster_country|origin_country_clean|roast_level_index|
+--------------------+-----------+--------------------+------------+-------------+------------+--------------+---------------+--------------------+-----------------+
|Colombia Finca Ca...|       94.0|Quindio Departmen...|       Light|         62.0|        82.0|         13.22|            USA|            Colombia|              1.0|
|Colombia Finca La...|       94.0|Quindio Departmen...|       Light|         63.0|        87.0|         10.14|            USA|            Colombia|              1.0|
|            In Bloom|       94.0|  Colombia, Ethiopia|       Light|         64.0|        88.0|          10.0|            USA|            Ethiopia|              1.0|
|Eth

In [36]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler (inputCols = ['total_score', 'agtron_ground', 'agtron_roast'], 
                                    outputCol = "Independent Features", 
                                    handleInvalid='skip')

output = featureassembler.transform(df_r)

output.select('Independent Features').show()

+--------------------+
|Independent Features|
+--------------------+
|    [94.0,62.0,82.0]|
|    [94.0,63.0,87.0]|
|    [94.0,64.0,88.0]|
|    [96.0,55.0,77.0]|
|    [95.0,62.0,78.0]|
|    [95.0,60.0,78.0]|
|    [95.0,60.0,78.0]|
|    [94.0,60.0,80.0]|
|    [94.0,58.0,77.0]|
|    [94.0,60.0,84.0]|
|    [94.0,60.0,86.0]|
|    [95.0,60.0,84.0]|
|    [95.0,62.0,84.0]|
|    [94.0,59.0,81.0]|
|    [94.0,62.0,82.0]|
|    [94.0,63.0,85.0]|
|    [96.0,60.0,77.0]|
|    [95.0,60.0,88.0]|
|    [95.0,65.0,87.0]|
|    [94.0,59.0,77.0]|
+--------------------+
only showing top 20 rows


In [37]:
finalized_data = output.select ("Independent Features", "price_per_100g")
finalized_data.show()

+--------------------+--------------+
|Independent Features|price_per_100g|
+--------------------+--------------+
|    [94.0,62.0,82.0]|         13.22|
|    [94.0,63.0,87.0]|         10.14|
|    [94.0,64.0,88.0]|          10.0|
|    [96.0,55.0,77.0]|        110.23|
|    [95.0,62.0,78.0]|        176.37|
|    [95.0,60.0,78.0]|          17.2|
|    [95.0,60.0,78.0]|        705.47|
|    [94.0,60.0,80.0]|        154.32|
|    [94.0,58.0,77.0]|        264.55|
|    [94.0,60.0,84.0]|         529.1|
|    [94.0,60.0,86.0]|         145.5|
|    [95.0,60.0,84.0]|          11.9|
|    [95.0,62.0,84.0]|         11.46|
|    [94.0,59.0,81.0]|          6.47|
|    [94.0,62.0,82.0]|         275.0|
|    [94.0,63.0,85.0]|         275.0|
|    [96.0,60.0,77.0]|         14.11|
|    [95.0,60.0,88.0]|          8.82|
|    [95.0,65.0,87.0]|         10.58|
|    [94.0,59.0,77.0]|          6.17|
+--------------------+--------------+
only showing top 20 rows


In [40]:
# delete null values
from pyspark.sql import functions as F

finalized_data = finalized_data.filter(
    F.col("price_per_100g").isNotNull()
)

In [42]:
from pyspark.ml.regression import LinearRegression
#train test split
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression (featuresCol = 'Independent Features', labelCol = 'price_per_100g')
regressor = regressor.fit(train_data)

In [44]:
regressor.coefficients
#Чим вищий total_score, тим більша ціна

DenseVector([49.6914, 3.4144, -1.7516])

In [45]:
regressor.intercept
#зсув?

-4664.454067218848

In [52]:
#Predication
pred_results = regressor.evaluate(test_data)
pred_results.predictions.show()
#або працювати з викидами, або пробувати інші моделі

+--------------------+--------------+------------------+
|Independent Features|price_per_100g|        prediction|
+--------------------+--------------+------------------+
|      [94.0,0.0,0.0]|          1.34| 6.541048909249184|
|    [94.0,40.0,49.0]|           2.0| 57.29068613545678|
|    [94.0,41.0,49.0]|           9.5|60.705125841448535|
|    [94.0,41.0,76.0]|          6.47|13.412173242195422|
|    [94.0,44.0,54.0]|        509.25|62.190490774377395|
|    [94.0,45.0,52.0]|         300.0| 69.10811215438753|
|    [94.0,45.0,67.0]|          5.07|42.834249599246505|
|    [94.0,46.0,53.0]|          2.86| 70.77096102337055|
|    [94.0,46.0,64.0]|          9.87| 51.50346181626719|
|    [94.0,47.0,56.0]|          0.44| 68.93062821833337|
|    [94.0,47.0,56.0]|          3.75| 68.93062821833337|
|    [94.0,47.0,69.0]|          5.29|46.159947337211634|
|    [94.0,48.0,54.0]|        127.87| 75.84824959834441|
|    [94.0,48.0,56.0]|         11.16| 72.34506792432512|
|    [94.0,48.0,59.0]|         

In [53]:
pred_results.r2, pred_results.meanAbsoluteError, pred_results.meanSquaredError
#лінійна регресія погано підходитьб є розбіжності та викиди 
#логарифмувати, або видалити викиди, або пробувати інші бібліотеки

(-0.04245536240564873, 133.0395456844549, 96963.67469985674)